In [ ]:
import pandas as pd
import numpy as np
import os
import mat73
import tabulate
from scipy import signal

In [ ]:
# Set the base dir to the folder including .mat files
base_dir = ''
list_of_files = os.listdir(base_dir)
list_of_files

In [ ]:
label_for_file_dictionary = {
    'struct_rs_R1.mat': 0,
    'struct_r1b_R1.mat': 1,
    'struct_r2b_R1.mat': 2,
    'struct_r3b_R1.mat': 3,
    'struct_r4b_R1.mat': 4
}

In [ ]:
def signal_divider(signal, window_len):
    
    fractions_of_signal = []
    starting_index = 0
    while starting_index < len(signal):
    
        fractions_of_signal.append(signal[starting_index : starting_index + window_len])
        starting_index = starting_index + window_len
    
    del fractions_of_signal[-1]
        
    return np.array(fractions_of_signal)

In [ ]:
# Set the exporting path for upsampled time domain signals
upsampled_dir = ''

#list_of_labels = ['Va', 'Vb', 'Vc', 'Ia', 'Ib', 'Ic', 'Vib_acpi', 'Vib_carc', 'Vib_acpe', 'Vib_axial', 'Vib_base']
list_of_labels = ['Ia', 'Ib', 'Ic', 'Vib_acpi', 'Vib_carc', 'Vib_acpe', 'Vib_axial', 'Vib_base']

len_of_vibrational_signals = 1024
desirable_upsampling_len_vibrational_signals = 153740

loads = ['torque05', 'torque10', 'torque15', 'torque20', 'torque25', 'torque30', 'torque35', 'torque40']
desirable_repeatitions = 10

for file in list_of_files:

    data_dict = mat73.loadmat(base_dir + file)
    main_key = list(data_dict.keys())[0]
    data_dict = data_dict[main_key]
    for load in loads:
        for rep in range(desirable_repeatitions):
            number_of_elec_points = len(data_dict[load][rep]['Va'])
            f_el_rate_to_f_vib = number_of_elec_points/desirable_upsampling_len_vibrational_signals
        
#            Va = pd.DataFrame(signal_divider(data_dict[load][rep]['Va'], int(f_el_rate_to_f_vib*len_of_vibrational_signals)))
#            Vb = pd.DataFrame(signal_divider(data_dict[load][rep]['Vb'], int(f_el_rate_to_f_vib*len_of_vibrational_signals)))
#            Vc = pd.DataFrame(signal_divider(data_dict[load][rep]['Vc'], int(f_el_rate_to_f_vib*len_of_vibrational_signals)))
            Ia = pd.DataFrame(signal_divider(data_dict[load][rep]['Ia'], int(f_el_rate_to_f_vib*len_of_vibrational_signals)))
            Ib = pd.DataFrame(signal_divider(data_dict[load][rep]['Ib'], int(f_el_rate_to_f_vib*len_of_vibrational_signals)))
            Ic = pd.DataFrame(signal_divider(data_dict[load][rep]['Ic'], int(f_el_rate_to_f_vib*len_of_vibrational_signals)))
        

            Vib_acpi = pd.DataFrame(signal_divider(
                signal.resample(data_dict[load][rep]['Vib_acpi'], desirable_upsampling_len_vibrational_signals), len_of_vibrational_signals))
            
            Vib_carc = pd.DataFrame(signal_divider(
                signal.resample(data_dict[load][rep]['Vib_carc'], desirable_upsampling_len_vibrational_signals),len_of_vibrational_signals))
            
            Vib_acpe = pd.DataFrame(signal_divider(
                signal.resample(data_dict[load][rep]['Vib_acpe'], desirable_upsampling_len_vibrational_signals), len_of_vibrational_signals))
            
            Vib_axial = pd.DataFrame(signal_divider(
                signal.resample(data_dict[load][rep]['Vib_axial'], desirable_upsampling_len_vibrational_signals), len_of_vibrational_signals))
            
            Vib_base = pd.DataFrame(signal_divider(
                signal.resample(data_dict[load][rep]['Vib_base'], desirable_upsampling_len_vibrational_signals), len_of_vibrational_signals))
                                    
#            list_of_dfs = [Va, Vb, Vc, Ia, Ib, Ic, Vib_acpi, Vib_carc, Vib_acpe, Vib_axial, Vib_base]
            list_of_dfs = [Ia, Ib, Ic, Vib_acpi, Vib_carc, Vib_acpe, Vib_axial, Vib_base]
    
            for i in range(len(list_of_dfs)):
                list_of_dfs[i].to_csv(upsampled_dir + list_of_labels[i] + '\\' + list_of_labels[i] + '_' + main_key + '_' + load + '_' + str(rep) + '.csv')
            
    print('{} is EXPORTED!'.format(file))
            